In [1]:
# !pip install -q \
#     transformers \
#     accelerate \
#     peft \
#     bitsandbytes \
#     datasets \
#     sentence-transformers \
#     faiss-cpu \
#     pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 93.4 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
import faiss
import json
import numpy as np
import fitz
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model

2025-12-17 10:31:33.609252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765967493.812932      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765967493.873149      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765967494.374391      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765967494.374432      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765967494.374435      55 computation_placer.cc:177] computation placer alr

In [3]:
MODEL_NAME = "Qwen/Qwen3-4B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,949,120 || all params: 4,025,417,216 || trainable%: 0.0733


In [5]:
dataset = load_dataset(
    "json",
    data_files="/kaggle/input/lora-train/lora_train.jsonl"
)

def tokenize(example):
    messages = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["response"]}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    out = tokenizer(
        text,
        truncation=True,
        max_length=512
    )
    out["labels"] = out["input_ids"].copy()
    return out

dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [6]:
args = TrainingArguments(
    output_dir="lora_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"]
)

trainer.train()

Step,Training Loss
10,1.912100
20,1.643300
30,1.599100
40,1.429400
50,1.493000
60,1.341800


TrainOutput(global_step=64, training_loss=1.562226191163063, metrics={'train_runtime': 179.1501, 'train_samples_per_second': 1.429, 'train_steps_per_second': 0.357, 'total_flos': 2236379911907328.0, 'train_loss': 1.562226191163063, 'epoch': 2.0})

In [7]:
model.save_pretrained("qwen3-4b-course")
tokenizer.save_pretrained("qwen3-4b-course")

('qwen3-4b-course/tokenizer_config.json',
 'qwen3-4b-course/special_tokens_map.json',
 'qwen3-4b-course/chat_template.jinja',
 'qwen3-4b-course/vocab.json',
 'qwen3-4b-course/merges.txt',
 'qwen3-4b-course/added_tokens.json',
 'qwen3-4b-course/tokenizer.json')

In [8]:
index = faiss.read_index("/kaggle/input/vectorstore/index.faiss")
texts = np.load("/kaggle/input/vectorstore/texts.npy", allow_pickle=True)

embedder = SentenceTransformer("intfloat/e5-base-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [9]:
def youtube_search(q):
    return f"https://www.youtube.com/results?search_query={q.replace(' ', '+')}"

def arxiv_search(q):
    return f"https://arxiv.org/search/?query={q.replace(' ', '+')}&searchtype=all"

In [10]:
def retrieve_context(question, k=4):
    emb = embedder.encode([question]).astype("float32")
    _, idx = index.search(emb, k)
    return "\n\n".join(texts[i] for i in idx[0])

In [16]:
def generate_answer(question):
    context = retrieve_context(question)

    messages = [
        {
            "role": "system",
            "content": "You are an IIIT course assistant."
        },
        {
            "role": "user",
            "content": f"""
Context:
{context}

Question:
{question}

Answer clearly, completely, and in a structured manner.
"""
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.6,
        top_p=0.95
    )

    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # ✅ ADD THESE
    yt_links = youtube_search(question)
    arxiv_links = arxiv_search(question)

    # Safety: ensure lists
    if isinstance(yt_links, str):
        yt_links = [yt_links]
    if isinstance(arxiv_links, str):
        arxiv_links = [arxiv_links]

    return {
        "answer": answer,
        "youtube": yt_links,
        "papers": arxiv_links
    }

# Validation

In [17]:
# 🔍 Trial test cell — verify RAG + LoRA model works end-to-end
test_question = "What is a random variable? Explain with examples."

print("=" * 70)
print(" IIIT Course Assistant – Trial RAG Test")
print("=" * 70)

print("\n🧑‍🎓 Question:")
print(test_question)

print("\n🔍 Thinking...\n")

result = generate_answer(test_question)

print("📘 Answer:")
print("-" * 70)
print(result["answer"].strip())
print("-" * 70)

if result.get("youtube"):
    print("\n📺 Suggested YouTube Videos:")
    for link in result["youtube"]:
        print("  •", link)

if result.get("papers"):
    print("\n📄 Relevant Research Papers:")
    for paper in result["papers"]:
        print("  •", paper)

print("\n" + "=" * 70)
print("✅ Trial test completed successfully")

 IIIT Course Assistant – Trial RAG Test

🧑‍🎓 Question:
What is a random variable? Explain with examples.

🔍 Thinking...

📘 Answer:
----------------------------------------------------------------------
system
You are an IIIT course assistant.
user

Context:
Continuous random variables
▶A random variable defined on R is discrete, if FX(·) is
piecewise constant.
▶A random variable defined on R is continuous, if FX(·) is a
continuous function.
▶Examples of Continuous random variables
1. Pick a number uniformly from [a, b].
2. Time interval between successive customers entering DMart.
3. Travel time from office to home.
4. Level of water in a dam or pending workload on a server.
8 / 29

Example of function of Random variables
▶Suppose X and Y are continuous independent random
variables. Let W = max(X, Y ) and Z = min(X, Y ) Find the
CDF and pdf of Z.
▶HW: When X and Y are exponential with parameters λ1 and
λ2 then Z is also exponential with parameter λ1 + λ2.
5 / 42

A running example
▶Con

# Test

In [13]:
doc = fitz.open("/kaggle/input/endsem/endsem.pdf")
questions = []

for page in doc:
    for line in page.get_text().split("\n"):
        if line.lower().startswith(("q", "question")):
            questions.append(line)

results = {}
for i, q in enumerate(questions):
    results[f"Q{i+1}"] = generate_answer(q)

with open("endsem_results.json", "w") as f:
    json.dump(results, f, indent=2)

FileNotFoundError: no such file: '/kaggle/input/endsem/endsem.pdf'